# 🧠 고전 CNN 모델 실습: LeNet-5 / VGG16

이번 노트북에서는 고전적인 CNN 모델 3종을 실습합니다.

1. **LeNet-5** — MNIST 직접 학습 및 평가
3. **VGG16** — ImageNet 사전학습 가중치로 예측 실습

모든 모델의 파라미터 수를 비교하고, 구조를 살펴봅니다.

In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils
from tensorflow.keras.applications import VGG16, MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np
import matplotlib.pyplot as plt
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.12.0


## [1] LeNet-5 구현 및 MNIST 학습

In [10]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print('Train:', x_train.shape, ' Test:', x_test.shape)

Train: (60000, 28, 28, 1)  Test: (10000, 28, 28, 1)


In [11]:
def build_lenet5():
    model = models.Sequential([
        layers.Conv2D(6, (5,5), activation='tanh', input_shape=(28,28,1), padding='same'),
        layers.AveragePooling2D(pool_size=(2,2)),
        layers.Conv2D(16, (5,5), activation='tanh'),
        layers.AveragePooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dense(120, activation='tanh'),
        layers.Dense(84, activation='tanh'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

lenet5 = build_lenet5()
lenet5.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 6)         156       
                                                                 
 average_pooling2d_2 (Averag  (None, 14, 14, 6)        0         
 ePooling2D)                                                     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 average_pooling2d_3 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                                     
                                                                 
 flatten_1 (Flatten)         (None, 400)               0         
                                                                 
 dense_3 (Dense)             (None, 120)              

In [12]:
history = lenet5.fit(x_train, y_train, epochs=5, batch_size=128, validation_split=0.1, verbose=2)

Epoch 1/5
422/422 - 6s - loss: 0.3708 - accuracy: 0.8931 - val_loss: 0.1453 - val_accuracy: 0.9573 - 6s/epoch - 14ms/step
Epoch 2/5
422/422 - 6s - loss: 0.1395 - accuracy: 0.9570 - val_loss: 0.0942 - val_accuracy: 0.9738 - 6s/epoch - 14ms/step
Epoch 3/5
422/422 - 4s - loss: 0.0891 - accuracy: 0.9729 - val_loss: 0.0729 - val_accuracy: 0.9785 - 4s/epoch - 10ms/step
Epoch 4/5
422/422 - 4s - loss: 0.0645 - accuracy: 0.9801 - val_loss: 0.0597 - val_accuracy: 0.9825 - 4s/epoch - 9ms/step
Epoch 5/5
422/422 - 4s - loss: 0.0507 - accuracy: 0.9845 - val_loss: 0.0541 - val_accuracy: 0.9857 - 4s/epoch - 10ms/step


In [13]:
test_loss, test_acc = lenet5.evaluate(x_test, y_test, verbose=0)
print(f"LeNet-5 테스트 정확도: {test_acc:.4f}")

LeNet-5 테스트 정확도: 0.9794


## [3] VGG16 (ImageNet 사전학습 가중치)

In [14]:
vgg16 = VGG16(weights='imagenet')
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [15]:
img_path = tf.keras.utils.get_file('elephant.jpg', 'https://cdn.hackaday.io/images/1219231493836853483.jpg')
img_vgg = image.load_img(img_path, target_size=(224, 224))
x_vgg = image.img_to_array(img_vgg)
x_vgg = np.expand_dims(x_vgg, axis=0)
x_vgg = preprocess_input(x_vgg)

preds_vgg = vgg16.predict(x_vgg)
print('VGG16 예측 결과:', decode_predictions(preds_vgg, top=3)[0])

1/1 [==============================] - 0s 157ms/step
VGG16 예측 결과: [('n02504458', 'African_elephant', 0.8400239), ('n01871265', 'tusker', 0.15768927), ('n02504013', 'Indian_elephant', 0.002286783)]


## [4] 모델별 파라미터 수 비교

In [8]:
models_info = {
    'LeNet-5 (MNIST)': lenet5.count_params(),
    'VGG16 (ImageNet)': vgg16.count_params()
}
for name, params in models_info.items():
    print(f'{name}: {params:,} parameters')

LeNet-5 (MNIST): 61,706 parameters
VGG16 (ImageNet): 138,357,544 parameters
